# Deformation map

This notebook will show how to compute a deformation map, i.e. $F_\text{calc}^\text{TAAM} - F_\text{calc}^\text{IAM}$.
We first load the structure information from a .cif file or a .pdb file. Here, a molecule in a $\text{P}2_12_12_1$ box is provided, but this can be replaced with whatever structure you have
Next, we initialize an IAM calculator and a TAAM calculator, set the resolution we want, and compute a deformation map. This is then saved in an .mtz-file.

In [ ]:
import pydiscamb
from cctbx import miller
from iotbx import pdb
from pathlib import Path

# Replace this with a path to your structure file
structure_file = Path().resolve() / "data" / "tyrosine.pdb"

# Load structure
xrs = pdb.input(file_name=str(structure_file)).xray_structure_simple()
xrs.scattering_type_registry(table="xray")

# Prepare calculators
iam_calculator = pydiscamb.DiscambWrapper(
    xrs,
    method=pydiscamb.FCalcMethod.IAM,
)
taam_calculator = pydiscamb.DiscambWrapper(
    xrs,
    method=pydiscamb.FCalcMethod.TAAM,
)

n_typed = sum(1 for type, lcs in taam_calculator.atom_type_assignment.values() if lcs)
n_total = len(taam_calculator.atom_type_assignment)
print(f"Ratio of atoms with assigned type: {n_typed / n_total :.1%}")

# Set resolution
miller_set = miller.build_set(
    crystal_symmetry=xrs.crystal_symmetry(), 
    anomalous_flag=False, 
    d_min=0.7,
)
iam_calculator.set_indices(miller_set.indices())
taam_calculator.set_indices(miller_set.indices())

# Compute structure factors
fcalc_iam = iam_calculator.f_calc()
fcalc_taam = taam_calculator.f_calc()

# Compute deformation map
scale = sum(map(lambda a, b: abs(a) * abs(b), fcalc_iam, fcalc_taam)) / sum(map(lambda a: abs(a)**2, fcalc_iam))
deformation_map = fcalc_taam - scale * fcalc_iam

# Create a miller.array to save
deformation_map_array = miller_set.array(
    data=deformation_map,
)
deformation_map_array.write_mtz(
    file_name="deformation_map.mtz",
)

Ratio of atoms with assigned type: 100.0%


1.020875100039912

Below is a capture from Coot, showing the resulting deformation map at 3.0 RMSD

![tyrosine_difference_map.png](./data/tyrosine_difference_map.png)


# Even easier
If the goal is simply to compute the deformation map at a given resolution, it can be performed even simpler:

In [7]:
w1 = pydiscamb.DiscambWrapper.from_file("data/tyrosine.pdb", pydiscamb.FCalcMethod.IAM)
w2 = pydiscamb.DiscambWrapper.from_file("data/tyrosine.pdb", pydiscamb.FCalcMethod.TAAM)

d_min = 1.5
fc1 = w1.f_calc(d_min)
fc2 = w2.f_calc(d_min)

deformation_map = fc2 - fc1